https://medium.datadriveninvestor.com/stop-loss-trailing-stop-or-take-profit-2-million-backtests-shed-light-dde23bda40be

# Define Parameters

In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import ccxt
import itertools
import ipywidgets
from datetime import datetime, timedelta
from numba import njit


In [2]:
seed = 42
symbols = ['ADAUSDT', 'BNBUSDT', 'BTCUSDT', 'EOSUSDT','ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'VETUSDT', 'XLMUSDT', 'XRPUSDT']
start_date = datetime(2017, 6, 1)
end_date = datetime(2023, 1, 21)
time_delta = end_date - start_date
window_len = timedelta(days=180)
window_count = 400
exit_types = ['SL', 'TS', 'TP', 'Random', 'Holding']
step = 0.01  # in %
stops = np.arange(step, 1 + step, step)

vbt.settings.plotting["layout"]["template"] = "vbt_dark"
vbt.settings.portfolio['freq'] = '1d'
vbt.settings.portfolio['init_cash'] = 100.  # in $
vbt.settings.portfolio['fees'] = 0.0025  # in %
vbt.settings.portfolio['slippage'] = 0.0025  # in %


In [3]:
print(pd.Series({
    'Start date': start_date,
    'End date': end_date,
    'Time period (days)': time_delta.days,
    'Assets': len(symbols),
    'Window length': window_len,
    'Windows': window_count,
    'Exit types': len(exit_types),
    'Stop values': len(stops),
    'Tests per asset': window_count * len(stops) * len(exit_types),
    'Tests per window': len(symbols) * len(stops) * len(exit_types),
    'Tests per exit type': len(symbols) * window_count * len(stops),
    'Tests per stop type and value': len(symbols) * window_count,
    'Tests total': len(symbols) * window_count * len(stops) * len(exit_types)
}))


Start date                       2017-06-01 00:00:00
End date                         2023-01-21 00:00:00
Time period (days)                              2060
Assets                                            10
Window length                      180 days, 0:00:00
Windows                                          400
Exit types                                         5
Stop values                                      100
Tests per asset                               200000
Tests per window                                5000
Tests per exit type                           400000
Tests per stop type and value                   4000
Tests total                                  2000000
dtype: object


In [4]:
# cols = ['Open', 'Low', 'High', 'Close', 'Volume']
# data = vbt.CCXTData.download(
#     symbols,
#     start=start_date,
#     end=end_date,
#     timeframe='1d',
#     exchange='binance'
#     )
# data.save('stop_sigs.pickle')
data = vbt.CCXTData.load('stop_sigs.pickle')
ohlcv_by_symbol = data.data


In [5]:
print(ohlcv_by_symbol.keys())
print(ohlcv_by_symbol['BTCUSDT'].shape)
ohlcv_by_symbol['BTCUSDT'].vbt.ohlcv.plot()


dict_keys(['ADAUSDT', 'BNBUSDT', 'BTCUSDT', 'EOSUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'VETUSDT', 'XLMUSDT', 'XRPUSDT'])
(1984, 5)


FigureWidget({
    'data': [{'close': array([ 4285.08,  4108.37,  4139.98, ..., 21071.59, 22667.21, 22783.55]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([ 4485.39,  4371.52,  4184.69, ..., 21192.  , 22755.93, 23371.8 ]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([ 4200.74,  3938.77,  3850.  , ..., 20659.19, 20861.28, 22422.  ]),
              'name': 'OHLC',
              'open': array([ 4261.48,  4285.08,  4108.37, ..., 20677.47, 21071.59, 22666.  ]),
              'type': 'ohlc',
              'uid': '94db60e8-c91f-45a3-9493-9013f4402897',
              'x': array([datetime.datetime(2017, 8, 17, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 18, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 19, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 1, 19, 0, 0, tzinfo=dat

In [6]:
ohlcv = data.concat()
print(ohlcv.keys())
ohlcv['Open'].shape

dict_keys(['Open', 'High', 'Low', 'Close', 'Volume'])


(1984, 10)

In [7]:
ohlcv['Open'].head(345)

symbol,ADAUSDT,BNBUSDT,BTCUSDT,EOSUSDT,ETHUSDT,LTCUSDT,TRXUSDT,VETUSDT,XLMUSDT,XRPUSDT
Open time,,,,,,,,,,
2017-08-17 00:00:00+00:00,NaN,NaN,4261.48,NaN,301.13,NaN,NaN,NaN,NaN,NaN
2017-08-18 00:00:00+00:00,NaN,NaN,4285.08,NaN,302.00,NaN,NaN,NaN,NaN,NaN
2017-08-19 00:00:00+00:00,NaN,NaN,4108.37,NaN,293.31,NaN,NaN,NaN,NaN,NaN
2017-08-20 00:00:00+00:00,NaN,NaN,4120.98,NaN,289.41,NaN,NaN,NaN,NaN,NaN
2017-08-21 00:00:00+00:00,NaN,NaN,4069.13,NaN,299.10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2018-07-23 00:00:00+00:00,0.16937,12.1594,7394.78,7.8461,457.47,82.20,0.03507,NaN,0.28345,0.44859
2018-07-24 00:00:00+00:00,0.16411,12.0348,7721.65,7.9336,450.05,81.43,0.03413,NaN,0.28600,0.44520
2018-07-25 00:00:00+00:00,0.17360,12.1469,8397.24,8.5809,479.24,89.02,0.03796,0.02250,0.30019,0.45741


In [8]:
ohlcv_data = {}
for k, v in ohlcv.items():
    v.dropna(inplace=True)
    ohlcv_data[k] = v.vbt.range_split(
        range_len=window_len.days, n=window_count)[0]
ohlcv_data['Open']


split_idx      0                                                       \
symbol     ADAUSDT  BNBUSDT  BTCUSDT EOSUSDT ETHUSDT LTCUSDT  TRXUSDT   
0          0.17360  12.1469  8397.24  8.5809  479.24   89.02  0.03796   
1          0.17274  13.0448  8175.63  8.5307  472.11   86.38  0.03717   
2          0.16397  13.3435  7920.00  8.2980  462.17   83.51  0.03586   
3          0.16586  13.9902  8188.57  8.4296  470.09   84.39  0.03656   
4          0.16387  14.6415  8225.04  8.3501  468.44   84.10  0.03693   
..             ...      ...      ...     ...     ...     ...      ...   
175        0.04199   5.8007  3553.06  2.3475  118.93   30.74  0.02396   
176        0.04420   6.0941  3591.84  2.4035  121.49   31.29  0.02482   
177        0.04470   6.3152  3613.32  2.4729  121.56   31.38  0.02491   
178        0.04373   6.4887  3594.87  2.4296  119.16   30.78  0.02440   
179        0.04515   6.5221  3665.75  2.4432  122.61   31.94  0.02418   

split_idx                              ...     399                            \
symbol      VETUSDT  XLMUSDT  XRPUSDT  ... ADAUSDT BNBUSDT   BTCUSDT EOSUSDT   
0          0.022500  0.30019  0.45741  ...  0.4752   244.7  21310.90   1.108   
1          0.021170  0.33400  0.45983  ...  0.4678   249.5  21254.67   1.111   
2          0.025030  0.30810  0.44878  ...  0.5112   271.6  22954.31   1.255   
3          0.025020  0.31890  0.45550  ...  0.5131   277.2  23845.25   1.313   
4          0.025730  0.31507  0.45643  ...  0.5219   293.6  23777.28   1.300   
..              ...      ...      ...  ...     ...     ...       ...     ...   
175        0.003849  0.10383  0.32154  ...  0.3505   299.0  21185.65   1.036   
176        0.003938  0.10555  0.32580  ...  0.3455   299.4  21132.29   1.022   
177        0.004185  0.10713  0.32468  ...  0.3264   286.7  20677.47   0.958   
178        0.004014  0.10561  0.32042  ...  0.3380   294.6  21071.59   0.981   
179        0.004069  0.10655  0.32672  ...  0.3651   305.1  22666.00   1.046   

split_idx                                                     
symbol     ETHUSDT LTCUSDT  TRXUSDT  VETUSDT XLMUSDT XRPUSDT  
0          1440.79   54.00  0.06370  0.02333  0.1057  0.3362  
1          1449.12   53.89  0.06537  0.02332  0.1057  0.3373  
2          1635.74   59.00  0.06866  0.02519  0.1126  0.3594  
3          1724.52   63.40  0.06968  0.02635  0.1167  0.3738  
4          1721.68   60.65  0.06924  0.02668  0.1178  0.3679  
..             ...     ...      ...      ...     ...     ...  
175        1576.94   85.94  0.06241  0.02134  0.0878  0.3855  
176        1565.56   87.04  0.06167  0.02127  0.0862  0.3868  
177        1511.44   82.62  0.05890  0.01970  0.0824  0.3782  
178        1551.09   84.21  0.06009  0.02031  0.0844  0.3929  
179        1658.53   90.28  0.06273  0.02237  0.0889  0.4129  

[180 rows x 4000 columns]

In [9]:
ohlcv_indexs = pd.Series(ohlcv['Open'].vbt.range_split(
    range_len=window_len.days, n=window_count)[1])
ohlcv_indexs


0      DatetimeIndex(['2018-07-25 00:00:00+00:00', '2...
1      DatetimeIndex(['2018-07-29 00:00:00+00:00', '2...
2      DatetimeIndex(['2018-08-01 00:00:00+00:00', '2...
3      DatetimeIndex(['2018-08-05 00:00:00+00:00', '2...
4      DatetimeIndex(['2018-08-09 00:00:00+00:00', '2...
                             ...                        
395    DatetimeIndex(['2022-07-11 00:00:00+00:00', '2...
396    DatetimeIndex(['2022-07-15 00:00:00+00:00', '2...
397    DatetimeIndex(['2022-07-19 00:00:00+00:00', '2...
398    DatetimeIndex(['2022-07-22 00:00:00+00:00', '2...
399    DatetimeIndex(['2022-07-26 00:00:00+00:00', '2...
Length: 400, dtype: object

In [10]:
print(ohlcv_data['Open'].columns)


MultiIndex([(  0, 'ADAUSDT'),
            (  0, 'BNBUSDT'),
            (  0, 'BTCUSDT'),
            (  0, 'EOSUSDT'),
            (  0, 'ETHUSDT'),
            (  0, 'LTCUSDT'),
            (  0, 'TRXUSDT'),
            (  0, 'VETUSDT'),
            (  0, 'XLMUSDT'),
            (  0, 'XRPUSDT'),
            ...
            (399, 'ADAUSDT'),
            (399, 'BNBUSDT'),
            (399, 'BTCUSDT'),
            (399, 'EOSUSDT'),
            (399, 'ETHUSDT'),
            (399, 'LTCUSDT'),
            (399, 'TRXUSDT'),
            (399, 'VETUSDT'),
            (399, 'XLMUSDT'),
            (399, 'XRPUSDT')],
           names=['split_idx', 'symbol'], length=4000)


In [11]:
entries = pd.DataFrame.vbt.signals.empty_like(ohlcv_data['Open'])
entries.iloc[0, :] = True
entries


split_idx     0                                                            \
symbol    ADAUSDT BNBUSDT BTCUSDT EOSUSDT ETHUSDT LTCUSDT TRXUSDT VETUSDT   
0            True    True    True    True    True    True    True    True   
1           False   False   False   False   False   False   False   False   
2           False   False   False   False   False   False   False   False   
3           False   False   False   False   False   False   False   False   
4           False   False   False   False   False   False   False   False   
..            ...     ...     ...     ...     ...     ...     ...     ...   
175         False   False   False   False   False   False   False   False   
176         False   False   False   False   False   False   False   False   
177         False   False   False   False   False   False   False   False   
178         False   False   False   False   False   False   False   False   
179         False   False   False   False   False   False   False   False   

split_idx                  ...     399                                  \
symbol    XLMUSDT XRPUSDT  ... ADAUSDT BNBUSDT BTCUSDT EOSUSDT ETHUSDT   
0            True    True  ...    True    True    True    True    True   
1           False   False  ...   False   False   False   False   False   
2           False   False  ...   False   False   False   False   False   
3           False   False  ...   False   False   False   False   False   
4           False   False  ...   False   False   False   False   False   
..            ...     ...  ...     ...     ...     ...     ...     ...   
175         False   False  ...   False   False   False   False   False   
176         False   False  ...   False   False   False   False   False   
177         False   False  ...   False   False   False   False   False   
178         False   False  ...   False   False   False   False   False   
179         False   False  ...   False   False   False   False   False   

split_idx                                          
symbol    LTCUSDT TRXUSDT VETUSDT XLMUSDT XRPUSDT  
0            True    True    True    True    True  
1           False   False   False   False   False  
2           False   False   False   False   False  
3           False   False   False   False   False  
4           False   False   False   False   False  
..            ...     ...     ...     ...     ...  
175         False   False   False   False   False  
176         False   False   False   False   False  
177         False   False   False   False   False  
178         False   False   False   False   False  
179         False   False   False   False   False  

[180 rows x 4000 columns]

https://vectorbt.dev/api/signals/generators/#vectorbt.signals.generators.OHLCSTX


In [12]:
sl_ohlcstx = vbt.OHLCSTX.run(
    entries,
    ohlcv_data["Open"],
    ohlcv_data["High"],
    ohlcv_data["Low"],
    ohlcv_data["Close"],
    sl_stop=list(stops),
)
sl_exits = sl_ohlcstx.exits.copy()
sl_close = sl_ohlcstx.close.copy()
sl_exits.vbt.rename_levels(
    {'ohlcstx_sl_stop': 'stop_value'}, strict=False, inplace=True)
sl_close.vbt.rename_levels(
    {'ohlcstx_sl_stop': 'stop_value'}, strict=False, inplace=True)


In [13]:
tsl_ohlcstx = vbt.OHLCSTX.run(
    entries,
    open=ohlcv_data["Open"],
    high=ohlcv_data["High"],
    low=ohlcv_data["Low"],
    close=ohlcv_data["Close"],
    sl_stop=list(stops),
    sl_trail=True
)
tsl_exits = tsl_ohlcstx.exits.copy()
tsl_close = tsl_ohlcstx.close.copy()
tsl_exits.vbt.rename_levels(
    {'ohlcstx_sl_stop': 'stop_value'}, strict=False, inplace=True)
tsl_close.vbt.rename_levels(
    {'ohlcstx_sl_stop': 'stop_value'}, strict=False, inplace=True)


In [14]:
tp_ohlcstx = vbt.OHLCSTX.run(
    entries,
    ohlcv_data["Open"],
    ohlcv_data["High"],
    ohlcv_data["Low"],
    ohlcv_data["Close"],
    tp_stop=list(stops),
)
tp_exits = tp_ohlcstx.exits.copy()
tp_close = tp_ohlcstx.close.copy()
tp_exits.vbt.rename_levels(
    {'ohlcstx_tp_stop': 'stop_value'}, strict=False, inplace=True)
tp_close.vbt.rename_levels(
    {'ohlcstx_tp_stop': 'stop_value'}, strict=False, inplace=True)


# Exploring number of exit signals

In [15]:
pd.Series({
    'SL': sl_exits.vbt.signals.total().mean(),
    'TS': tsl_exits.vbt.signals.total().mean(),
    'TP': tp_exits.vbt.signals.total().mean(),
}, name='avg-num-signals')


SL    0.384333
TS    0.585745
TP    0.597065
Name: avg-num-signals, dtype: float64

In [16]:
pd.DataFrame({
    'Stop Loss': sl_exits.vbt.signals.total().groupby('stop_value').mean(),
    'Trailing Stop': tsl_exits.vbt.signals.total().groupby('stop_value').mean(),
    'Take Profit': tp_exits.vbt.signals.total().groupby('stop_value').mean()
}).vbt.plot(xaxis_title='Stop value', yaxis_title='Avg number of signals')


FigureWidget({
    'data': [{'name': 'Stop Loss',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3aa1b12e-1a4e-4a3f-a446-46a0afde5678',
              'x': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                          0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                          0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                          0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
                          0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
                          0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72,
                          0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84,
                          0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,
                          0.97, 0.98, 

In [17]:
sl_exits.iloc[-1,:] = True
tsl_exits.iloc[-1,:] = True
tp_exits.iloc[-1,:] = True

sl_exits = sl_exits.vbt.signals.first(reset_by=entries, allow_gaps=True)
tsl_exits = tsl_exits.vbt.signals.first(reset_by=entries, allow_gaps=True)
tp_exits = tp_exits.vbt.signals.first(reset_by=entries, allow_gaps=True)

pd.Series({
    'SL': sl_exits.vbt.signals.total().mean(),
    'TS': tsl_exits.vbt.signals.total().mean(),
    'TP': tp_exits.vbt.signals.total().mean(),
}, name='avg-num-signals')

SL    1.0
TS    1.0
TP    1.0
Name: avg-num-signals, dtype: float64

In [18]:
hold_exits = pd.DataFrame.vbt.signals.empty_like(sl_exits)
hold_exits.iloc[-1,:] = True
hold_exits.shape

(180, 400000)

In [19]:
rand_exits = hold_exits.vbt.shuffle(seed=seed)
rand_exits.shape

(180, 400000)

In [20]:
exits = pd.DataFrame.vbt.concat(
    sl_exits,
    tsl_exits,
    tp_exits,
    rand_exits,
    hold_exits,
    keys=pd.Index(exit_types, name="exit_type")
)
exits.shape

(180, 2000000)

In [21]:
avg_distance = entries.vbt.signals.between_ranges(other=exits).duration.mean(
).groupby(['exit_type', 'stop_value']).mean().unstack(level='exit_type')
avg_distance.mean()


exit_type
Holding    179.000000
Random      89.432010
SL         129.012518
TP         101.413803
TS         102.999323
dtype: float64

In [22]:
avg_distance

exit_type,Holding,Random,SL,TP,TS
stop_value,,,,,
0.01,179.0,89.84300,11.62700,13.52225,1.33200
0.02,179.0,88.62725,16.09500,16.77075,1.48050
0.03,179.0,88.42225,20.32825,19.67575,1.72600
0.04,179.0,88.91975,23.98400,23.52100,2.04925
0.05,179.0,89.42825,27.35275,27.36700,2.53950
...,...,...,...,...,...
0.96,179.0,89.25275,179.00000,141.88350,179.00000
0.97,179.0,89.00275,179.00000,142.31975,179.00000
0.98,179.0,88.38600,179.00000,142.64500,179.00000


In [23]:
avg_distance[exit_types].vbt.plot(
    xaxis_title='Stop Value', yaxis_title='Avg distance to entry'
)

FigureWidget({
    'data': [{'name': 'SL',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6ef497f6-2176-462f-be79-587e5efbcfff',
              'x': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                          0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                          0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                          0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
                          0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
                          0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72,
                          0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84,
                          0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,
                          0.97, 0.98, 0.99, 1

# Run Simulation

In [24]:
pf = vbt.Portfolio.from_signals(
    ohlcv_data['Close'],
    entries,
    exits
)

In [25]:
len(pf.orders)

3995570

In [26]:
total_return = pf.total_return()
total_return.shape

(2000000,)

# Analyze performance

In [27]:
total_return_by_type = total_return.unstack(level='exit_type')[exit_types]
total_return_by_type['Holding'].describe(percentiles=[])

count    400000.000000
mean          0.605318
std           2.129665
min          -0.836152
50%          -0.029204
max          21.849741
Name: Holding, dtype: float64

In [28]:
total_return_by_type['Holding'].vbt.histplot(
    xaxis_title='Total return',
    xaxis_tickformat='.2%',
    yaxis_title='Count',
    trace_kwargs=dict(marker_color=vbt.settings["plotting"]["color_schema"]["purple"])
)

FigureWidget({
    'data': [{'marker': {'color': '#9467bd'},
              'name': 'Holding',
              'opacity': 1,
              'showlegend': True,
              'type': 'histogram',
              'uid': '2b97777f-ccee-4de9-9c61-0fca53f920df',
              'x': array([-0.75430718, -0.51270986, -0.57140193, ..., -0.07448174, -0.15245736,
                           0.18441902])}],
    'layout': {'barmode': 'overlay',
               'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700,
               'xaxis': {'tickformat': '.2%', 'title': {'text': 'Total return'}},
               'yaxis': {'title': {'text': 'Count'}}}
})

In [29]:
pd.DataFrame({
    'Mean' : total_return_by_type.mean(),
    'Median' : total_return_by_type.median(),
    'Std' : total_return_by_type.std(),
})

,Mean,Median,Std
exit_type,,,
SL,0.487120,-0.101755,2.014147
TS,0.373893,-0.050665,1.703968
TP,0.126223,0.129385,0.459372
Random,0.265038,-0.011830,1.272439
Holding,0.605318,-0.029204,2.129665


In [30]:
total_return_by_type.vbt.boxplot(
    trace_kwargs=dict(boxpoints=False),
    yaxis_title='Total return',
    yaxis_tickformat='.2%'
)

FigureWidget({
    'data': [{'boxpoints': False,
              'name': 'SL',
              'showlegend': True,
              'type': 'box',
              'uid': '07a08c78-f6d9-4058-9d20-cf4583efee90',
              'y': array([-0.06009743, -0.07510948, -0.04090756, ..., -0.07448174, -0.15245736,
                           0.18441902])},
             {'boxpoints': False,
              'name': 'TS',
              'showlegend': True,
              'type': 'box',
              'uid': 'b1054a3a-4994-402d-8fea-91486cf1f414',
              'y': array([-0.06009743,  0.06519731, -0.04090756, ..., -0.07448174, -0.15245736,
                           0.18441902])},
             {'boxpoints': False,
              'name': 'TP',
              'showlegend': True,
              'type': 'box',
              'uid': '20c3b336-ba17-4b7c-8376-d46bc02871e5',
              'y': array([-0.06009743,  0.01467773, -0.57140193, ..., -0.07448174, -0.15245736,
                           0.18441902])},
             

In [31]:
(total_return_by_type > 0 ).mean().rename('Win Rate')

exit_type
SL         0.386560
TS         0.427233
TP         0.631232
Random     0.480517
Holding    0.480250
Name: Win Rate, dtype: float64

# aggregate by stop type & value

In [32]:
init_cash = vbt.settings.portfolio['init_cash']

def get_expectancy(total_return_by_type, level_name):
    grouped = total_return_by_type.groupby(level_name, axis=0)
    win_rate = grouped.apply(lambda x: (x>0).mean())
    avg_win = grouped.apply(lambda x: init_cash * x[x>0].mean()).fillna(0)
    avg_loss = grouped.apply(lambda x: init_cash * x[x<0].mean()).fillna(0)
    return win_rate * avg_win - (1 - win_rate) * np.abs(avg_loss)

expectancy_by_stop = get_expectancy(total_return_by_type, 'stop_value')

expectancy_by_stop.mean()

exit_type
SL         48.711983
TS         37.389349
TP         12.622302
Random     26.353141
Holding    60.531753
dtype: float64

In [33]:
expectancy_by_stop.vbt.plot(
    xaxis_title='Stop Value',
    yaxis_title='Expectancy'
)

FigureWidget({
    'data': [{'name': 'SL',
              'showlegend': True,
              'type': 'scatter',
              'uid': '808092b9-9803-45be-b994-59aedec3bc8c',
              'x': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                          0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                          0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                          0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
                          0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
                          0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72,
                          0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84,
                          0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,
                          0.97, 0.98, 0.99, 1

In [34]:
return_values = np.sort(total_return_by_type['Holding'].values)
idxs = np.ceil(np.linspace(0, len(return_values)-1, 21)).astype(int)
bins = return_values[idxs][:-1]

def bin_return(total_return_by_type):
    classes = pd.cut(total_return_by_type['Holding'], bins=bins, right=True)
    new_level = pd.Index(np.array(classes.apply(lambda x: x.right)), name='bin_right')
    return total_return_by_type.vbt.stack_index(new_level, axis=0)

binned_total_return_by_type = bin_return(total_return_by_type)
expectancy_by_bin = get_expectancy(binned_total_return_by_type, 'bin_right')

expectancy_by_bin.vbt.plot(
    trace_kwargs=dict(mode='lines'),
    xaxis_title='Total return of holding',
    xaxis_tickformat='.2%',
    yaxis_title='Expectancy'
)


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'SL',
              'showlegend': True,
              'type': 'scatter',
              'uid': '1604e25d-0662-406d-9cf8-1c81e8373dd4',
              'x': array([-0.606 , -0.543 , -0.475 , -0.416 , -0.352 , -0.286 , -0.224 , -0.169 ,
                          -0.102 , -0.0291,  0.047 ,  0.142 ,  0.285 ,  0.478 ,  0.67  ,  0.94  ,
                           1.361 ,  1.994 ,  3.902 ]),
              'y': array([-44.93889456, -41.49191854, -39.13146576, -36.0563687 , -32.63126247,
                          -29.12519541, -25.84328745, -22.05331798, -17.65806679, -12.96196786,
                           -7.37508422,  -1.15216994,   7.98251944,  21.41891096,  37.57126821,
                           61.26243489,  88.49206239, 130.07659134, 237.78886633])},
             {'mode': 'lines',
              'name': 'TS',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'fffc6fea-9918-4104-bc53-45

# Bonus: Jupyter dashboard

pip install kaleido==0.1.0post1

In [35]:
range_starts = pd.DatetimeIndex(list(map(lambda x: x[0], ohlcv_indexs)))
range_ends = pd.DatetimeIndex(list(map(lambda x: x[-1], ohlcv_indexs)))

symbol_lvl = total_return_by_type.index.get_level_values('symbol')
split_lvl = total_return_by_type.index.get_level_values('split_idx')
range_start_lvl = range_starts[split_lvl]
range_end_lvl = range_ends[split_lvl]

asset_multi_select = ipywidgets.SelectMultiple(
    options=symbols,
    value=symbols,
    rows=len(symbols),
    description='Symbols'
)
dates = np.unique(ohlcv['Open'].index)
date_range_slider = ipywidgets.SelectionRangeSlider(
    options=dates,
    index=(0, len(dates)-1),
    orientation="horizontal",
    readout=False,
    continuous_update=False
)
range_start_label = ipywidgets.Label()
range_end_label = ipywidgets.Label()
metric_dropdown = ipywidgets.Dropdown(
    options=["Mean", "Median", "Win Rate", "Expectancy"],
    value="Expectancy"
)

stop_scatter = vbt.plotting.Scatter(
    trace_names=exit_types,
    x_labels=stops, 
    xaxis_title='Stop value', 
    yaxis_title='Expectancy'
)
stop_scatter_img = ipywidgets.Image(
    format='png',
    width=stop_scatter.fig.layout.width,
    height=stop_scatter.fig.layout.height
)
bin_scatter = vbt.plotting.Scatter(
    trace_names=exit_types,
    x_labels=expectancy_by_bin.index, 
    trace_kwargs=dict(mode='lines'),
    xaxis_title='Total return of holding',
    xaxis_tickformat='%',
    yaxis_title='Expectancy'
)
bin_scatter_img = ipywidgets.Image(
    format='png',
    width=bin_scatter.fig.layout.width,
    height=bin_scatter.fig.layout.height
)

In [36]:
def update_scatter(*args, **kwargs):
    _symbols = asset_multi_select.value
    _from = date_range_slider.value[0]
    _to = date_range_slider.value[1]
    _metric_name = metric_dropdown.value
    
    range_mask = (range_start_lvl >= _from) & (range_end_lvl <= _to)
    asset_mask = symbol_lvl.isin(_symbols)
    filt = total_return_by_type[range_mask & asset_mask]
    
    filt_binned = bin_return(filt)
    if _metric_name == "Mean":
        filt_metric = filt.groupby("stop_value").mean()
        filt_bin_metric = filt_binned.groupby("bin_right").mean()
    elif _metric_name == "Median":
        filt_metric = filt.groupby("stop_value").median()
        filt_bin_metric = filt_binned.groupby("bin_right").median()
    elif _metric_name == "Win Rate":
        filt_metric = (filt > 0).groupby("stop_value").mean()
        filt_bin_metric = (filt_binned > 0).groupby("bin_right").mean()
    elif _metric_name == "Expectancy":
        filt_metric = get_expectancy(filt, "stop_value")
        filt_bin_metric = get_expectancy(filt_binned, "bin_right")
        
    stop_scatter.fig.update_layout(yaxis_title=_metric_name)
    stop_scatter.update(filt_metric)
    stop_scatter_img.value = stop_scatter.fig.to_image(format="png")
    
    bin_scatter.fig.update_layout(yaxis_title=_metric_name)
    bin_scatter.update(filt_bin_metric)
    bin_scatter_img.value = bin_scatter.fig.to_image(format="png")
    
    range_start_label.value = np.datetime_as_string(_from.to_datetime64(), unit="D")
    range_end_label.value = np.datetime_as_string(_to.to_datetime64(), unit="D")
    
asset_multi_select.observe(update_scatter, names="value")
date_range_slider.observe(update_scatter, names="value")
metric_dropdown.observe(update_scatter, names="value")
update_scatter()

In [37]:
dashboard = ipywidgets.VBox([
    asset_multi_select,
    ipywidgets.HBox([
        range_start_label,
        date_range_slider,
        range_end_label
    ]),
    metric_dropdown,
    stop_scatter_img,
    bin_scatter_img
])
dashboard